In [153]:
#!/usr/bin/env python
from __future__ import print_function
from itertools import count

import torch
import torch.autograd
import torch.nn.functional as F
from torch.autograd import Variable

POLY_DEGREE = 4
W_target = torch.randn(POLY_DEGREE, 1) * 5
b_target = torch.randn(1) * 5


def make_features(x):
    """Builds features i.e. a matrix with columns [x, x^2, x^3, x^4]."""
    x = x.unsqueeze(1)
    return torch.cat([x ** i for i in range(1, POLY_DEGREE+1)], 1)


def f(x):
    """Approximated function."""
    return x.mm(W_target) + b_target[0]


def poly_desc(W, b):
    """Creates a string description of a polynomial."""
    result = 'y = '
    for i, w in enumerate(W):
        result += '{:+.2f} x^{} '.format(w, len(W) - i)
    result += '{:+.2f}'.format(b[0])
    return result


def get_batch(batch_size=32):
    """Builds a batch i.e. (x, f(x)) pair."""
    random = torch.randn(batch_size)
    x = make_features(random)
    y = f(x)
    return Variable(x), Variable(y)


# Define model
fc = torch.nn.Linear(W_target.size(0), 1)

# Define optimizer
optimizer = torch.optim.Adam(fc.parameters(), lr=0.1, betas=(0.5, 0.999), eps=1e-08, weight_decay=0.0000)

for batch_idx in count(1):
    # Get data
    batch_x, batch_y = get_batch()

    # Reset gradients
    fc.zero_grad()

    # Forward pass
    output = F.smooth_l1_loss(fc(batch_x), batch_y)
    loss = output.data[0]

    # Backward pass
    output.backward()

    # Apply gradients
    optimizer.step()
    
    # Stop criterion
    if loss < 1e-3:
        break

print('Loss: {:.6f} after {} batches'.format(loss, batch_idx))
print('==> Learned function:\t' + poly_desc(fc.weight.data.view(-1), fc.bias.data))
print('==> Actual function:\t' + poly_desc(W_target.view(-1), b_target))



 1.1023e+00  1.2150e+00  1.3393e+00  1.4763e+00
-6.5593e-01  4.3024e-01 -2.8221e-01  1.8511e-01
-3.4385e-01  1.1823e-01 -4.0654e-02  1.3979e-02
-1.1943e+00  1.4264e+00 -1.7035e+00  2.0345e+00
 5.7706e-01  3.3299e-01  1.9216e-01  1.1089e-01
 1.5147e-01  2.2944e-02  3.4755e-03  5.2645e-04
 4.2011e-01  1.7649e-01  7.4144e-02  3.1149e-02
 4.5940e-01  2.1105e-01  9.6959e-02  4.4543e-02
 7.4260e-01  5.5145e-01  4.0951e-01  3.0410e-01
 2.7738e-01  7.6942e-02  2.1342e-02  5.9200e-03
-1.2260e+00  1.5031e+00 -1.8427e+00  2.2592e+00
 1.2402e+00  1.5380e+00  1.9073e+00  2.3654e+00
 6.9641e-01  4.8499e-01  3.3775e-01  2.3521e-01
-7.5748e-01  5.7377e-01 -4.3462e-01  3.2922e-01
-6.2369e-01  3.8899e-01 -2.4261e-01  1.5131e-01
 4.6365e-01  2.1497e-01  9.9669e-02  4.6211e-02
 7.3202e-01  5.3586e-01  3.9226e-01  2.8715e-01
-1.2899e-03  1.6639e-06 -2.1462e-09  2.7684e-12
 6.4400e-01  4.1474e-01  2.6709e-01  1.7201e-01
-1.1506e+00  1.3239e+00 -1.5233e+00  1.7527e+00
-6.0310e-01  3.6372e-01 -2.1936e-01  1.